<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_amarillo@4x.png?raw=true" alt="esquema" />
</div>


# Laboratorio ETL: Análisis del Sistema Energético en España

## Objetivo

Durante todos los laboratorios de esta semana realizarás un proceso completo de ETL para analizar la relación entre la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de un año. Además, complementarán este análisis con datos demográficos y económicos extraídos del Instituto Nacional de Estadística (INE). El **objetivo principal** del análisis es **examinar cómo la demanda, el consumo y la generación eléctrica en diferentes provincias de España a lo largo de los años están influenciados por factores demográficos y económicos, como la población y el PIB provincial**. El análisis busca identificar patrones y correlaciones entre estas variables para comprender mejor las dinámicas energéticas regionales y su relación con el desarrollo socioeconómico en España.

Antes de realizar el análisis, vamos a definir las hipótesis con las que vamos a trabajar, las cuales definirán todo tu análisis y planteamiento de los laboratorios: 

- **Hipótesis 1: La demanda eléctrica está correlacionada con la población de la provincia.** Provincias con mayor población tienden a tener una mayor demanda eléctrica.
  
- **Hipótesis 2: El crecimiento económico (medido por el PIB) está correlacionado con el consumo eléctrico.** Las provincias con un PIB más alto o en crecimiento experimentan un mayor consumo de energía.

- **Hipótesis 3: La proporción de generación renovable está relacionada con factores económicos o geográficos.** Provincias con un mayor desarrollo económico o con condiciones geográficas favorables (como más horas de sol o viento) tienden a generar más energía renovable.


## Tareas Laboratorio Transformación

En este laboratorio, tu objetivo será limpiar y preparar los datos extraídos previamente de diferentes fuentes para su posterior análisis. Trabajarás con datos provenientes de la API de Red Eléctrica Española (REE) y del Instituto Nacional de Estadística (INE). Estos datos incluyen información sobre demanda y generación eléctrica a nivel provincial, así como datos demográficos y económicos. 


- Cargar los Datos Extraídos:

  - **Demanda Eléctrica:** Carga los datos de demanda eléctrica extraídos de la API de REE.

  - **Generación Eléctrica:** Carga los datos de generación eléctrica diferenciados por tipo de energía (eólica, solar, hidroeléctrica, etc.) a nivel provincial.

  - **Datos Demográficos:** Carga los datos demográficos por provincia extraídos de la web del INE.

  - **Datos Económicos:** Carga los datos del PIB por provincia obtenidos del INE.


-Limpieza de Datos:

- Datos de la API de REE:

  - **Demanda Eléctrica:**

    - **Conversión de Timestamps:** Asegúrate de que las fechas estén correctamente formateadas en `datetime`. Si es necesario, convierte los datos a un formato uniforme (por ejemplo, `YYYY-MM` para datos mensuales).

    - **Tratamiento de Valores Nulos:** Identifica y maneja los valores nulos en caso de que los haya. Puedes optar por eliminar filas con valores faltantes.

    - **Estandarización de Nombres de Provincias:** Verifica que los nombres de las provincias estén estandarizados y coincidan en todos los conjuntos de datos. Si hay inconsistencias, corrígelas.

  - **Generación Eléctrica:**

    - **Desagregación de Tecnologías:** Asegúrate de que los datos estén correctamente desglosados por tipo de energía. Revisa que los campos correspondientes a energía eólica, solar, hidroeléctrica, etc., estén bien identificados y sin errores.

    - **Normalización de Unidades:** Verifica que las unidades de energía estén estandarizadas (por ejemplo, MWh). Realiza las conversiones necesarias si se encuentran en otras unidades.

    - **Identificación de Outliers:** Revisa los valores extremos o atípicos en la generación de energía y decide si deben ser tratados o eliminados.

- Datos del INE:

  - **Datos Demográficos:**

    - **Consistencia en la Codificación de Provincias:** Asegúrate de que los nombres de las provincias en los datos demográficos coincidan con los nombres utilizados en los datos eléctricos.

    - **Revisión de Categorías:** Verifica que las categorías de edad, sexo, y nacionalidad estén correctamente etiquetadas y sean consistentes en todo el dataset.

    - **Manejo de Valores Faltantes:** Revisa la presencia de valores faltantes y decide cómo tratarlos (relleno, eliminación o sustitución).

  - **Datos Económicos:**

    - **Normalización del PIB:** Si los datos del PIB están en diferentes unidades o escalas, asegúrate de normalizarlos para que sean comparables entre provincias.

    - **Agrupación Temporal:** Si los datos económicos están disponibles en diferentes periodos temporales, agrúpalos y normalízalos para que coincidan con los datos eléctricos en términos de granularidad temporal (mensual o anual).

NOTA: Ten en cuenta que los datos los vamos a tener que insertar en una base de datos mañana, por lo que toda esta limpieza os recomendamos que la penséis para poder crear e insertar los datos mañana. 

In [1]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Chrome


from time import sleep
# Selenium para establecer la configuración del driver
# -----------------------------------------------------------------------
from selenium import webdriver

# Para generar una barra de proceso en los bucles for
# -----------------------------------------------------------------------
from tqdm import tqdm

# Para trabajar con ficheros
# -----------------------------------------------------------------------
import os

import re

import zipfile
import shutil

import pandas as pd
pd.options.display.float_format = '{:20,.2f}'.format

import requests
import json

/Users/davidfranco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## Red Eléctrica Generación

In [2]:
years = [2019, 2020, 2021]

cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [3]:
df_completo_generacion = pd.DataFrame()

for year in years:
    for comunidad, id in cod_comunidades.items():
        try:
            archivo_path = f"/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/generacion/{year}/{comunidad}.json"
            #print(archivo_path)
            with open(archivo_path, 'r', encoding="utf-8") as archivo:
                datos = json.load(archivo)
            
            lista_datos = []

            for entry in datos["included"]:
                tipo = entry["type"]
                id_tipo = entry["id"]
                #descripcion = entry["attributes"].get("description")
                #actualizacion = entry["attributes"].get("last-update")
                
                for value_entry in entry["attributes"]["values"]:
                    lista_datos.append({
                        "ID Comunidad": id,
                        "Tipo": tipo,
                        #"ID": id_tipo,
                        #"Descripción": descripcion,
                        #"Última actualización": actualizacion,
                        "Mes": value_entry["datetime"],
                        "Año":year,
                        "Valor": value_entry["value"],
                        "Porcentaje": value_entry["percentage"]
                    })
            
            df = pd.DataFrame(lista_datos)
            
            df_completo_generacion = pd.concat([df_completo_generacion, df], ignore_index=True)
        except:
            pass

In [4]:
df_completo_generacion[["Mes","Resto"]] = df_completo_generacion["Mes"].str.split("T", expand = True)
df_completo_generacion.drop(columns="Resto", inplace=True)
df_completo_generacion["Mes"] = pd.to_datetime(df_completo_generacion["Mes"], format='%Y-%m-%d').dt.month

In [5]:
df_completo_generacion

,ID Comunidad,Tipo,Mes,Año,Valor,Porcentaje
0,8745,Solar fotovoltaica,1,2019,5.05,0.01
1,8745,Solar fotovoltaica,2,2019,5.53,0.01
2,8745,Solar fotovoltaica,3,2019,6.56,0.01
3,8745,Solar fotovoltaica,4,2019,7.92,0.02
4,8745,Solar fotovoltaica,5,2019,8.68,0.02
...,...,...,...,...,...,...
3552,20,Generación renovable,8,2021,"82,996.98",1.00
3553,20,Generación renovable,9,2021,"58,651.08",1.00
3554,20,Generación renovable,10,2021,"72,563.02",1.00
3555,20,Generación renovable,11,2021,"161,015.09",1.00


In [6]:
df_completo_generacion.dtypes

ID Comunidad      int64
Tipo             object
Mes               int32
Año               int64
Valor           float64
Porcentaje      float64
dtype: object

In [7]:
df_completo_generacion.isna().value_counts()

ID Comunidad  Tipo   Mes    Año    Valor  Porcentaje
False         False  False  False  False  False         3557
Name: count, dtype: int64

In [8]:
# Eliminamos porque hace referencia al total de energías renovables
df_completo_generacion = df_completo_generacion[df_completo_generacion['Tipo'] != 'Generación renovable']


In [9]:
df_tipos_energia = pd.DataFrame([df_completo_generacion["Tipo"].unique()])
df_tipos_energia = df_tipos_energia.T.reset_index()
df_tipos_energia.rename(columns={"index": "ID Energía", 0: "Tipo Energía"}, inplace=True)
df_tipos_energia

,ID Energía,Tipo Energía
0,0,Solar fotovoltaica
1,1,Residuos renovables
2,2,Hidráulica
3,3,Eólica
4,4,Solar térmica
5,5,Otras renovables
6,6,Hidroeólica


In [10]:
df_tipos_energia.to_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosFinales/tipos_energia.csv")

In [11]:
df_completo_generacion

,ID Comunidad,Tipo,Mes,Año,Valor,Porcentaje
0,8745,Solar fotovoltaica,1,2019,5.05,0.01
1,8745,Solar fotovoltaica,2,2019,5.53,0.01
2,8745,Solar fotovoltaica,3,2019,6.56,0.01
3,8745,Solar fotovoltaica,4,2019,7.92,0.02
4,8745,Solar fotovoltaica,5,2019,8.68,0.02
...,...,...,...,...,...,...
3540,20,Otras renovables,8,2021,645.33,0.01
3541,20,Otras renovables,9,2021,695.54,0.01
3542,20,Otras renovables,10,2021,588.32,0.01
3543,20,Otras renovables,11,2021,434.94,0.00


In [12]:
df_completo_generacion = df_completo_generacion.merge(df_tipos_energia, left_on="Tipo", right_on="Tipo Energía")
df_completo_generacion = df_completo_generacion.drop(columns = ["Tipo","Tipo Energía"])

In [13]:
df_completo_generacion = df_completo_generacion.reindex(columns = ["Año", "Mes", "ID Comunidad", "ID Energía", "Valor","Porcentaje"])

In [14]:
df_completo_generacion.columns

Index(['Año', 'Mes', 'ID Comunidad', 'ID Energía', 'Valor', 'Porcentaje'], dtype='object')

In [15]:
df_completo_generacion = df_completo_generacion[['ID Energía', 'ID Comunidad', 'Año', 'Mes', 'Valor', 'Porcentaje']]
df_completo_generacion

,ID Energía,ID Comunidad,Año,Mes,Valor,Porcentaje
0,0,8745,2019,1,5.05,0.01
1,0,8745,2019,2,5.53,0.01
2,0,8745,2019,3,6.56,0.01
3,0,8745,2019,4,7.92,0.02
4,0,8745,2019,5,8.68,0.02
...,...,...,...,...,...,...
2904,5,20,2021,8,645.33,0.01
2905,5,20,2021,9,695.54,0.01
2906,5,20,2021,10,588.32,0.01
2907,5,20,2021,11,434.94,0.00


In [16]:
df_completo_generacion.to_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosFinales/generacion.csv")

### Función outliers

In [17]:
def identificar_outliers(df, columna):
    """
    Identifica outliers en una columna de un DataFrame utilizando el método IQR.
    
    Parámetros:
    df (DataFrame): El DataFrame que contiene la columna a evaluar.
    columna (str): El nombre de la columna a evaluar.

    Retorna:
    DataFrame: Un DataFrame que contiene solo los outliers.
    """
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[columna] < lower_bound) | (df[columna] > upper_bound)]
    return outliers


## Comunidades

In [18]:
df_comunidades = pd.DataFrame([cod_comunidades]).T.reset_index()
df_comunidades.rename(columns={"index": "Comunidad", 0: "ID Comunidad"}, inplace=True)
df_comunidades = df_comunidades.reindex(columns=["ID Comunidad", "Comunidad"])
df_comunidades

,ID Comunidad,Comunidad
0,8744,Ceuta
1,8745,Melilla
2,4,Andalucía
3,5,Aragón
4,6,Cantabria
5,7,Castilla - La Mancha
6,8,Castilla y León
7,9,Cataluña
8,10,País Vasco
9,11,Principado de Asturias


In [19]:
df_comunidades.to_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosFinales/comunidades.csv")

## Provincias

In [20]:
provincias_comunidades = {
    'Albacete': 7,
    'Alicante/Alacant': 15,
    'Almería': 4,
    'Araba/Álava': 10,
    'Asturias': 11,
    'Ávila': 8,
    'Badajoz': 16,
    'Balears, Illes': 8743,
    'Barcelona': 9,
    'Bizkaia': 10,
    'Burgos': 8,
    'Cáceres': 16,
    'Cádiz': 4,
    'Cantabria': 6,
    'Castellón/Castelló': 15,
    'Ciudad Real': 7,
    'Córdoba': 4,
    'Coruña, A': 17,
    'Cuenca': 7,
    'Gipuzkoa': 10,
    'Girona': 9,
    'Granada': 4,
    'Guadalajara': 7,
    'Huelva': 4,
    'Huesca': 5,
    'Jaén': 4,
    'León': 8,
    'Lleida': 9,
    'Lugo': 17,
    'Madrid': 13,
    'Málaga': 4,
    'Murcia': 21,
    'Navarra': 14,
    'Ourense': 17,
    'Palencia': 8,
    'Palmas, Las': 8742,
    'Pontevedra': 17,
    'Rioja, La': 20,
    'Salamanca': 8,
    'Santa Cruz de Tenerife': 8742,
    'Segovia': 8,
    'Sevilla': 4,
    'Soria': 8,
    'Tarragona': 9,
    'Teruel': 5,
    'Toledo': 7,
    'Valencia/València': 15,
    'Valladolid': 8,
    'Zamora': 8,
    'Zaragoza': 5,
    'Ceuta': 8744,
    'Melilla': 8745
}

In [21]:
df_provincias = pd.DataFrame([provincias_comunidades]).T.reset_index()
df_provincias.rename(columns = {"index":"Provincia",0:"ID Comunidad"})
df_provincias.head(2)

,index,0
0,Albacete,7
1,Alicante/Alacant,15


## Relación Provincias Comunidades

In [22]:
provincias_comunidades = {
    (2, 'Albacete'): 7,  # Castilla - La Mancha
    (3, 'Alicante/Alacant'): 15,  # Comunitat Valenciana
    (4, 'Almería'): 4,  # Andalucía
    (1, 'Araba/Álava'): 10,  # País Vasco
    (33, 'Asturias'): 11,  # Principado de Asturias
    (5, 'Ávila'): 8,  # Castilla y León
    (6, 'Badajoz'): 16,  # Extremadura
    (7, 'Balears, Illes'): 8743,  # Illes Balears
    (8, 'Barcelona'): 9,  # Cataluña
    (48, 'Bizkaia'): 10,  # País Vasco
    (9, 'Burgos'): 8,  # Castilla y León
    (10, 'Cáceres'): 16,  # Extremadura
    (11, 'Cádiz'): 4,  # Andalucía
    (39, 'Cantabria'): 6,  # Cantabria
    (12, 'Castellón/Castelló'): 15,  # Comunitat Valenciana
    (13, 'Ciudad Real'): 7,  # Castilla - La Mancha
    (14, 'Córdoba'): 4,  # Andalucía
    (15, 'Coruña, A'): 17,  # Galicia
    (16, 'Cuenca'): 7,  # Castilla - La Mancha
    (20, 'Gipuzkoa'): 10,  # País Vasco
    (17, 'Girona'): 9,  # Cataluña
    (18, 'Granada'): 4,  # Andalucía
    (19, 'Guadalajara'): 7,  # Castilla - La Mancha
    (21, 'Huelva'): 4,  # Andalucía
    (22, 'Huesca'): 5,  # Aragón
    (23, 'Jaén'): 4,  # Andalucía
    (24, 'León'): 8,  # Castilla y León
    (25, 'Lleida'): 9,  # Cataluña
    (27, 'Lugo'): 17,  # Galicia
    (28, 'Madrid'): 13,  # Comunidad de Madrid
    (29, 'Málaga'): 4,  # Andalucía
    (30, 'Murcia'): 21,  # Región de Murcia
    (31, 'Navarra'): 14,  # Comunidad Foral de Navarra
    (32, 'Ourense'): 17,  # Galicia
    (34, 'Palencia'): 8,  # Castilla y León
    (35, 'Palmas, Las'): 8742,  # Canarias
    (36, 'Pontevedra'): 17,  # Galicia
    (26, 'Rioja, La'): 20,  # La Rioja
    (37, 'Salamanca'): 8,  # Castilla y León
    (38, 'Santa Cruz de Tenerife'): 8742,  # Canarias
    (40, 'Segovia'): 8,  # Castilla y León
    (41, 'Sevilla'): 4,  # Andalucía
    (42, 'Soria'): 8,  # Castilla y León
    (43, 'Tarragona'): 9,  # Cataluña
    (44, 'Teruel'): 5,  # Aragón
    (45, 'Toledo'): 7,  # Castilla - La Mancha
    (46, 'Valencia/València'): 15,  # Comunitat Valenciana
    (47, 'Valladolid'): 8,  # Castilla y León
    (49, 'Zamora'): 8,  # Castilla y León
    (50, 'Zaragoza'): 5,  # Aragón
    (51, 'Ceuta'): 8744,  # Ceuta
    (52, 'Melilla'): 8745,  # Melilla
}

In [23]:
df_provincias_comunidades = pd.DataFrame([(prov[0], prov[1], comunidad) for prov, comunidad in provincias_comunidades.items()],
                  columns=["id_provincia", "provincia", "id_ccaa"])
df_provincias_comunidades.head(5)

,id_provincia,provincia,id_ccaa
0,2,Albacete,7
1,3,Alicante/Alacant,15
2,4,Almería,4
3,1,Araba/Álava,10
4,33,Asturias,11


In [24]:
df_provincias_comunidades.to_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosFinales/provincias.csv")

## Red Eléctrica Demanda

In [25]:
df_completo_demanda = pd.DataFrame()

for year in years:
    for comunidad, id in cod_comunidades.items():
        try:
            archivo_path = f"/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/demanda/{year}/{comunidad}.json"
            #print(archivo_path)
            with open(archivo_path, 'r', encoding="utf-8") as archivo:
                datos = json.load(archivo)
            
            lista_datos = []

            for entry in datos["included"]:
                
                for value_entry in entry["attributes"]["values"]:
                    lista_datos.append({
                        "ID Comunidad": id,
                        "Mes": value_entry["datetime"],
                        "Año":year,
                        "Valor": value_entry["value"],
                    })
            
            df = pd.DataFrame(lista_datos)
            
            df_completo_demanda = pd.concat([df_completo_demanda, df], ignore_index=True)
        except:
            pass

In [26]:
df_completo_demanda[["Mes","Resto"]] = df_completo_demanda["Mes"].str.split("T", expand = True)
df_completo_demanda.drop(columns="Resto", inplace=True)
df_completo_demanda["Mes"] = pd.to_datetime(df_completo_demanda["Mes"], format='%Y-%m-%d').dt.month

In [27]:
df_completo_demanda = df_completo_demanda.reindex(columns=["Año","Mes","ID Comunidad","Valor"])
df_completo_demanda

,Año,Mes,ID Comunidad,Valor
0,2019,1,8744,"24,566,154.55"
1,2019,2,8744,"21,280,529.20"
2,2019,3,8744,"21,935,545.64"
3,2019,4,8744,"20,692,209.65"
4,2019,5,8744,"21,134,292.13"
...,...,...,...,...
679,2021,8,20,"22,059,793.28"
680,2021,9,20,"20,972,956.06"
681,2021,10,20,"20,197,477.97"
682,2021,11,20,"21,433,492.69"


In [28]:
df_completo_demanda.isna().value_counts()

Año    Mes    ID Comunidad  Valor
False  False  False         False    684
Name: count, dtype: int64

In [29]:
df_completo_demanda.dtypes

Año               int64
Mes               int32
ID Comunidad      int64
Valor           float64
dtype: object

In [30]:
df_completo_demanda.to_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosFinales/demanda.csv")

In [31]:
df_suma = df_completo_generacion[((df_completo_generacion["Año"] == 2019) & df_completo_generacion["Mes"] == 1) & (df_completo_generacion["ID Comunidad"] == 8745) & (df_completo_generacion["ID Energía"] == 0)]["Valor"].sum()
df_suma

np.float64(40.05)

In [32]:
df_nuevo_porc = df_completo_generacion[((df_completo_generacion["Año"] == 2019) & df_completo_generacion["Mes"] == 1) & (df_completo_generacion["ID Comunidad"] == 8745) & (df_completo_generacion["ID Energía"] == 0)]["Valor"]/df_suma

In [33]:
df_nuevo_porc

0                    0.13
2                    0.16
4                    0.22
6                    0.20
8                    0.17
10                   0.12
Name: Valor, dtype: float64

In [34]:
#df_suma = df_completo_generacion[((df_completo_generacion["Año"] == {year}) & df_completo_generacion["Mes"] == {month}) & (df_completo_generacion["ID Comunidad"] == {id}) & (df_completo_generacion["ID Energía"] == {tipo})]["Valor"].sum()

#df_nuevo_porc = df_completo_generacion[((df_completo_generacion["Año"] == {year}) & df_completo_generacion["Mes"] == {month}) & (df_completo_generacion["ID Comunidad"] == {id}) & (df_completo_generacion["ID Energía"] == {tipo})]["Valor"]/df_suma

In [35]:
for year in years:
    for month in range(1, 13):  # Assuming months should be 1 to 12
        for id, comunidad in cod_comunidades.items():  # Assuming cod_comunidades is a dictionary
            for tipo in range(0, 7):
                df_suma = df_completo_generacion[
                    (df_completo_generacion["Año"] == year) &
                    (df_completo_generacion["Mes"] == month) &
                    (df_completo_generacion["ID Comunidad"] == id) &
                    (df_completo_generacion["ID Energía"] == tipo)
                ]["Valor"].sum()
                
                df_completo_generacion.loc[
                    (df_completo_generacion["Año"] == year) &
                    (df_completo_generacion["Mes"] == month) &
                    (df_completo_generacion["ID Comunidad"] == id) &
                    (df_completo_generacion["ID Energía"] == tipo), "Nuevo Porcentaje"
                ] = round(df_completo_generacion[
                    (df_completo_generacion["Año"] == year) &
                    (df_completo_generacion["Mes"] == month) &
                    (df_completo_generacion["ID Comunidad"] == id) &
                    (df_completo_generacion["ID Energía"] == tipo)
                ]["Valor"] / df_suma, 5)


In [36]:
df_completo_generacion

,ID Energía,ID Comunidad,Año,Mes,Valor,Porcentaje,Nuevo Porcentaje
0,0,8745,2019,1,5.05,0.01,NaN
1,0,8745,2019,2,5.53,0.01,NaN
2,0,8745,2019,3,6.56,0.01,NaN
3,0,8745,2019,4,7.92,0.02,NaN
4,0,8745,2019,5,8.68,0.02,NaN
...,...,...,...,...,...,...,...
2904,5,20,2021,8,645.33,0.01,NaN
2905,5,20,2021,9,695.54,0.01,NaN
2906,5,20,2021,10,588.32,0.01,NaN
2907,5,20,2021,11,434.94,0.00,NaN


In [37]:
df_completo_generacion.dtypes

ID Energía            int64
ID Comunidad          int64
Año                   int64
Mes                   int32
Valor               float64
Porcentaje          float64
Nuevo Porcentaje    float64
dtype: object

## IPC Población

In [38]:
df_poblacion = pd.read_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosDescargados/poblacion.csv", sep=';', encoding='utf-8')

In [39]:
df_poblacion

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2022,47.475.420
1,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2021,47.385.107
2,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2020,47.450.795
3,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2019,47.026.208
4,TOTAL ESPAÑA,TOTAL EDADES,TOTAL,Ambos sexos,2018,46.722.980
...,...,...,...,...,...,...
63595,52 Melilla,65 y más,% Extranjeros,Mujeres,2002,"5,13"
63596,52 Melilla,65 y más,% Extranjeros,Mujeres,2001,"4,65"
63597,52 Melilla,65 y más,% Extranjeros,Mujeres,2000,"3,34"
63598,52 Melilla,65 y más,% Extranjeros,Mujeres,1999,"0,81"


In [40]:
sucio = df_poblacion[
    (df_poblacion["Edad (3 grupos de edad)"] != "TOTAL EDADES") & 
    (df_poblacion["Españoles/Extranjeros"] != "TOTAL") & 
    (df_poblacion["Provincias"] != "TOTAL ESPAÑA") &
    (df_poblacion["Españoles/Extranjeros"] != "% Extranjeros") &
    (df_poblacion["Sexo"] != "Ambos sexos")
]

In [41]:
sucio['Provincias'] = sucio['Provincias'].apply(lambda x: "".join(re.findall(pattern= r"\d+", string=x))).astype(int)

/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_86741/1966336634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucio['Provincias'] = sucio['Provincias'].apply(lambda x: "".join(re.findall(pattern= r"\d+", string=x))).astype(int)


In [42]:
sucio.reset_index(inplace=True, drop=True)

In [43]:
sucio.head(1)

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,2,0-15 años,Españoles,Hombres,2022,26.845


In [44]:
provincias = sucio["Provincias"].unique()
codigos = dict()
for provincia in provincias:
    splits = provincia.split()
    codigos["".join(re.findall(pattern= r" (\D+)", string=provincia))] = splits[0]

codigos

AttributeError: 'numpy.int64' object has no attribute 'split'

In [ ]:
lista_tuplas_provincias = []
for k, v in codigos.items():
    lista_tuplas_provincias.append(tuple([int(v),k]))

In [53]:
sucio["Total"] = sucio["Total"].str.replace(".","").astype(int)

/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_86741/4012148157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucio["Total"] = sucio["Total"].str.replace(".","").astype(int)


In [55]:
sucio.dtypes

Provincias                  int64
Edad (3 grupos de edad)    object
Españoles/Extranjeros      object
Sexo                       object
Año                         int64
Total                       int64
dtype: object

In [56]:
sucio

,Provincias,Edad (3 grupos de edad),Españoles/Extranjeros,Sexo,Año,Total
0,2,0-15 años,Españoles,Hombres,2022,26845
1,2,0-15 años,Españoles,Hombres,2021,27304
2,2,0-15 años,Españoles,Hombres,2020,27979
3,2,0-15 años,Españoles,Hombres,2019,28513
4,2,0-15 años,Españoles,Hombres,2018,28881
...,...,...,...,...,...,...
15595,52,65 y más,Extranjeros,Mujeres,2002,222
15596,52,65 y más,Extranjeros,Mujeres,2001,193
15597,52,65 y más,Extranjeros,Mujeres,2000,136
15598,52,65 y más,Extranjeros,Mujeres,1999,30


In [57]:
sucio.to_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosFinales/poblacion.csv")

In [ ]:
df_grupos_edades = pd.DataFrame(sucio["Edad (3 grupos de edad)"].unique())

In [ ]:
df_grupos_edades.reset_index(inplace=True)
df_grupos_edades = df_grupos_edades.rename(columns={"index":"id_grupo",0:"Grupo edad"})
df_grupos_edades

,id_grupo,Grupo edad
0,0,0-15 años
1,1,16-64 años
2,2,65 y más


In [ ]:
df_grupos_edades.to_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosFinales/edades.csv")

## INE PIB

In [ ]:
df_pib = pd.read_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosDescargados/pib.csv", sep=';', encoding='latin-1')

In [ ]:
df_pib

,Provincias,Ramas de actividad,periodo,Total
0,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2021(P),8.663.814
1,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2020,7.855.556
2,02 Albacete,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2019,8.475.070
3,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",2021(P),884.324
4,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",2020,768.976
...,...,...,...,...
1711,52 Melilla,Impuestos netos sobre los productos,2020,132.490
1712,52 Melilla,Impuestos netos sobre los productos,2019,151.213
1713,52 Melilla,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2021(P),1.616.543
1714,52 Melilla,PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO,2020,1.514.009


In [ ]:
sucio2 = df_pib[
    (df_pib["Ramas de actividad"] != "PRODUCTO INTERIOR BRUTO A PRECIOS DE MERCADO")
]

sucio2[["Año","Resto"]] = sucio2["periodo"].str.split("(", expand = True)
sucio2 = sucio2.drop(columns = ["periodo", "Resto"])

/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_83897/3596973143.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucio2[["Año","Resto"]] = sucio2["periodo"].str.split("(", expand = True)
/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_83897/3596973143.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sucio2[["Año","Resto"]] = sucio2["periodo"].str.split("(", expand = True)


In [ ]:
sucio2

,Provincias,Ramas de actividad,Total,Año
3,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",884.324,2021
4,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",768.976,2020
5,02 Albacete,"A. Agricultura, ganadería, silvicultura y pesca",791.464,2019
6,02 Albacete,"B_E. Industrias extractivas, industria manufac...",1.397.006,2021
7,02 Albacete,"B_E. Industrias extractivas, industria manufac...",1.194.438,2020
...,...,...,...,...
1708,52 Melilla,Valor añadido bruto total,1.381.519,2020
1709,52 Melilla,Valor añadido bruto total,1.473.873,2019
1710,52 Melilla,Impuestos netos sobre los productos,153.994,2021
1711,52 Melilla,Impuestos netos sobre los productos,132.490,2020


In [ ]:
sucio2['Provincias'] = sucio2['Provincias'].apply(lambda x: "".join(re.findall(pattern= r"\d+", string=x))).astype(int)

In [ ]:
sucio2.reset_index(inplace=True,drop=True)

In [ ]:
sucio2 = sucio2.reindex(columns= ["Provincias", "Ramas de actividad", "Año", "Total"])

In [ ]:
sucio2

,Provincias,Ramas de actividad,Año,Total
0,2,"A. Agricultura, ganadería, silvicultura y pesca",2021,884.324
1,2,"A. Agricultura, ganadería, silvicultura y pesca",2020,768.976
2,2,"A. Agricultura, ganadería, silvicultura y pesca",2019,791.464
3,2,"B_E. Industrias extractivas, industria manufac...",2021,1.397.006
4,2,"B_E. Industrias extractivas, industria manufac...",2020,1.194.438
...,...,...,...,...
1399,52,Valor añadido bruto total,2020,1.381.519
1400,52,Valor añadido bruto total,2019,1.473.873
1401,52,Impuestos netos sobre los productos,2021,153.994
1402,52,Impuestos netos sobre los productos,2020,132.490


In [ ]:
sucio2["Ramas de actividad"].value_counts()

Ramas de actividad
A. Agricultura, ganadería, silvicultura y pesca                                                                                                                                                                                                         156
B_E. Industrias extractivas, industria manufacturera, suministro de energía eléctrica, gas, vapor y aire acondicionado, suministro de agua, actividades de saneamiento, gestión de residuos y descontaminación                                          156
C. - De las cuales: Industria manufacturera                                                                                                                                                                                                             156
F. Construcción                                                                                                                                                                                                                  

In [ ]:
df_pib_total = sucio2[
    (sucio2["Ramas de actividad"] == "Valor añadido bruto total") | 
    (sucio2["Ramas de actividad"] == "Impuestos netos sobre los productos")
]
df_pib_total["Año"] = df_pib_total["Año"].astype(int)
df_pib_total

/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_83897/4111556232.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pib_total["Año"] = df_pib_total["Año"].astype(int)


,Provincias,Ramas de actividad,Año,Total
21,2,Valor añadido bruto total,2021,7.838.487
22,2,Valor añadido bruto total,2020,7.168.120
23,2,Valor añadido bruto total,2019,7.686.471
24,2,Impuestos netos sobre los productos,2021,825.327
25,2,Impuestos netos sobre los productos,2020,687.436
...,...,...,...,...
1399,52,Valor añadido bruto total,2020,1.381.519
1400,52,Valor añadido bruto total,2019,1.473.873
1401,52,Impuestos netos sobre los productos,2021,153.994
1402,52,Impuestos netos sobre los productos,2020,132.490


In [ ]:
df_pib_total["Total"] = df_pib_total["Total"].str.replace(".","").astype(int)

/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_83897/2920250510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pib_total["Total"] = df_pib_total["Total"].str.replace(".","").astype(int)


In [ ]:
df_pib_total.dtypes

Provincias             int64
Ramas de actividad    object
Año                    int64
Total                  int64
dtype: object

In [ ]:
df_pib_total.to_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosFinales/pib_total.csv")

In [ ]:
df_pib_final = sucio2[
    (sucio2["Ramas de actividad"] != "Valor añadido bruto total") & 
    (sucio2["Ramas de actividad"] != "Impuestos netos sobre los productos")
]

In [ ]:
df_pib_final

,Provincias,Ramas de actividad,Año,Total
0,2,"A. Agricultura, ganadería, silvicultura y pesca",2021,884.324
1,2,"A. Agricultura, ganadería, silvicultura y pesca",2020,768.976
2,2,"A. Agricultura, ganadería, silvicultura y pesca",2019,791.464
3,2,"B_E. Industrias extractivas, industria manufac...",2021,1.397.006
4,2,"B_E. Industrias extractivas, industria manufac...",2020,1.194.438
...,...,...,...,...
1393,52,"K_N. Actividades financieras y de seguros, act...",2020,254.596
1394,52,"K_N. Actividades financieras y de seguros, act...",2019,258.800
1395,52,"O_U. Administración pública y defensa, segurid...",2021,783.628
1396,52,"O_U. Administración pública y defensa, segurid...",2020,759.737


In [ ]:
df_pib_final["Año"] = df_pib_final["Año"].astype(int)

/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_83897/589133067.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pib_final["Año"] = df_pib_final["Año"].astype(int)


In [ ]:
df_pib_final["Total"] = df_pib_final["Total"].str.replace(".","").astype(int)

/var/folders/yj/y1bv8zk12j11bnjq1hc6x_d80000gn/T/ipykernel_83897/731472290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pib_final["Total"] = df_pib_final["Total"].str.replace(".","").astype(int)


In [ ]:
df_pib_final.dtypes

Provincias             int64
Ramas de actividad    object
Año                    int64
Total                  int64
dtype: object

In [ ]:
df_pib_final.to_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosFinales/pib.csv")

array(['A. Agricultura, ganadería, silvicultura y pesca',
       'B_E. Industrias extractivas, industria manufacturera, suministro de energía eléctrica, gas, vapor y aire acondicionado, suministro de agua, actividades de saneamiento, gestión de residuos y descontaminación',
       'C. - De las cuales: Industria manufacturera', 'F. Construcción',
       'G_J. Comercio al por mayor y al por menor, reparación de vehículos de motor y motocicletas, transporte y almacenamiento, hostelería, información y comunicaciones',
       'K_N. Actividades financieras y de seguros, actividades inmobiliarias, actividades profesionales, científicas y técnicas, actividades administrativas y servicios auxiliares',
       'O_U. Administración pública y defensa, seguridad social obligatoria, educación, actividades sanitarias y de servicios sociales, actividades artísticas, recreativas y de entretenimiento, reparación de artículos de uso doméstico y otros servicios'],
      dtype=object)

In [ ]:
df_ramas = pd.DataFrame(df_pib_final["Ramas de actividad"].unique())

In [ ]:
df_ramas.reset_index(inplace=True)
df_ramas = df_ramas.rename(columns={"index":"id_rama",0:"nombre_rama"})
df_ramas

,id_rama,nombre_rama
0,0,"A. Agricultura, ganadería, silvicultura y pesca"
1,1,"B_E. Industrias extractivas, industria manufac..."
2,2,C. - De las cuales: Industria manufacturera
3,3,F. Construcción
4,4,"G_J. Comercio al por mayor y al por menor, rep..."
5,5,"K_N. Actividades financieras y de seguros, act..."
6,6,"O_U. Administración pública y defensa, segurid..."


In [ ]:
df_ramas.to_csv("/Users/davidfranco/Library/CloudStorage/OneDrive-Personal/Hackio/Jupyter/Clase 1028 - Extracción/laboratorio-modulo5-leccion01-etl-extraccion/datos/DatosFinales/ramas.csv")